In [1]:
import sys
import os

# Step 1: Get current directory 
current_directory = os.getcwd()        # '/home/bear-b/users/jshi008/IvyProjects/FIDLA/train_WaLeF_models'

# Step 2: Get path of parent directory (one level up)
parent_directory = os.path.dirname(current_directory)    # '/home/bear-b/users/jshi008/IvyProjects/FIDLA'

# Step 3: Append parent directory to sys.path
sys.path.append(parent_directory)

In [2]:
import torch
from torch import nn
import torch.utils.data as data_utils
import time
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ExponentialLR

from IPython import display


from baselines.cnn import CNN
from preprocess.BaselinePrerocess import baseline_process

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Preprocess

In [7]:
# ====== preprocessing parameters ======
n_hours = 72
K = 24 
masked_value = 1e-10
split_1 = 0.8
split_2 = 0.9

train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = baseline_process(n_hours, K, masked_value, split_1, split_2)

train_X_mask, val_X_mask, test_X_mask, train_ws_y, val_ws_y, test_ws_y = torch.from_numpy(train_X_mask), torch.from_numpy(val_X_mask), torch.from_numpy(test_X_mask), torch.from_numpy(train_ws_y), torch.from_numpy(val_ws_y), torch.from_numpy(test_ws_y)

# train_X_mask = train_X_mask.permute(0, 2, 1)
# val_X_mask = val_X_mask.permute(0, 2, 1)
# test_X_mask = test_X_mask.permute(0, 2, 1)
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)


torch.Size([77069, 96, 16]) torch.Size([9634, 96, 16]) torch.Size([19268, 96, 16]) torch.Size([77069, 96]) torch.Size([9634, 96]) torch.Size([19268, 96])


In [8]:
batch_size = 512
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_X_mask, train_ws_y), batch_size=batch_size, shuffle=True)
val_loader = data_utils.DataLoader(data_utils.TensorDataset(val_X_mask, val_ws_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_X_mask, test_ws_y), batch_size=batch_size, shuffle=True)

# Model

- dont know how incorporate L1L2 regularization into  the linear layer

In [17]:
# ===== model parameters ======
mlp_unit1 = 64
mlp_unit2 = 16
dropout = 0.1
l1_reg = 1e-5
l2_reg = 1e-5
learning_rate = 1e-3
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 500
EPOCHS = 3000
BATCH = 512
input_shape = train_X_mask.shape[1:]

In [16]:
train_X_mask.shape[1:]

torch.Size([96, 16])

In [19]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__(self, input_shape, mlp_unit1, mlp_unit2, l1_reg, l2_reg, dropout, masked_value):
        super(MLP, self).__init__()
        
        self.masked_value = masked_value
        
        self.dense1 = nn.Linear(input_shape[1], mlp_unit1)
        self.dropout1 = nn.Dropout(dropout)
        
        self.dense2 = nn.Linear(mlp_unit1, mlp_unit2)
        self.dropout2 = nn.Dropout(dropout)
        
        self.flatten = nn.Flatten()
        self.output_layer = nn.Linear(input_shape[0] * mlp_unit2, 96)
        
        self.l1_reg = l1_reg
        self.l2_reg = l2_reg

    def forward(self, x):
        # x = x.masked_fill(x == self.masked_value, 0)
        
        x = F.relu(self.dense1(x))
        x = self.dropout1(x)
        
        x = F.relu(self.dense2(x))
        x = self.dropout2(x)
        # print("Dense 2: ", x.shape)
        

        x = self.flatten(x)
        # print("flatten: ", x.shape)
        x = self.output_layer(x)
        
        return x


inp = torch.rand([1, 96, 16]).to(device)

model = MLP(input_shape, mlp_unit1, mlp_unit2, l1_reg, l2_reg, dropout, masked_value).to(device)
out = model(inp)
print(out.shape)

torch.Size([1, 96])


In [ ]:
import torch
from torch.nn.parameter import UninitializedParameter

def count_parameters(model):
    dummy_input = torch.randn(1, 16, 96)
    model.forward(dummy_input)
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad and not isinstance(p, UninitializedParameter))

total_params = count_parameters(cnn)
print(total_params)